In [1]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import unicodedata
import re

In [2]:
data=pd.read_excel('赛题1测试数据集/专利.xlsx',index_col=0)

In [3]:
data

,专利类型,授权公告日,申请日
企业编号,,,
4084,发明公布,2010-01-29,2010-09-01
4084,发明公布,2010-01-22,2010-07-14
4084,发明公布,2014-12-31,2015-05-13
4084,发明公布,2013-11-26,2014-02-19
4084,发明公布,2013-12-04,2014-03-26
4084,发明授权,2013-10-30,2017-01-04
4084,发明公布,2010-01-22,2010-06-30
4084,发明公布,2013-11-28,2014-02-12
4084,发明授权,2010-02-09,2013-04-24


In [4]:
data.drop_duplicates(keep='first',inplace=True)

In [5]:
data=data.replace(['发明公布更正','实用新型更正','外观设计更正','发明授权更正'],np.nan)
data=data.dropna()
count=pd.DataFrame(data.index.value_counts()).T.reindex(columns=list(range(4001,4526))).fillna(0)
count.index=pd.Index(['专利_count'])
data_1=pd.get_dummies(data['专利类型'],prefix='专利类型')
data_1=data_1.groupby('企业编号').sum().T

In [6]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    _ = re.findall(r'\d.+\d', cell)
    if not _:
        return None
    cell = _[0]
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [7]:
data_2=data[['授权公告日']].applymap(to_day).groupby('企业编号').mean().T
data_3=data[['申请日']].applymap(to_day).groupby('企业编号').mean().T

In [8]:
data=pd.concat([data_1,data_2,data_3,count])
data=data.reindex(columns=list(range(4001,4526))).fillna(0)
data.to_pickle('专利-testing.pickle')
data

,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
专利类型_发明专利,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,5.000000
专利类型_发明公布,77.000000,200.000000,9.00,10.000000,83.000000,16.000000,62.000000,52.000000,31.000000,18.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.000000,650.000000
专利类型_发明授权,74.000000,166.000000,6.00,1.000000,73.000000,6.000000,49.000000,12.000000,36.000000,16.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,738.000000
专利类型_外观设计,1.000000,15.000000,0.00,0.000000,28.000000,10.000000,24.000000,28.000000,2.000000,4.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,271.000000
专利类型_实用新型,15.000000,37.000000,5.00,70.000000,283.000000,53.000000,73.000000,51.000000,2.000000,18.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.000000,337.000000
授权公告日,-1661.610778,-2948.361244,-1837.45,-1652.456790,-3310.436831,-2551.435294,-1711.769231,-2319.447552,-1721.464789,-1782.910714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-997.269841,-3147.038481
申请日,-1210.077844,-2395.899522,-1475.60,-1358.358025,-2929.822270,-2286.705882,-1408.514423,-1982.524476,-1253.915493,-1368.285714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-746.063492,-2278.461269
专利_count,167.000000,418.000000,20.00,81.000000,467.000000,85.000000,208.000000,143.000000,71.000000,56.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.000000,2001.000000


In [9]:
data.index

Index(['专利类型_发明专利', '专利类型_发明公布', '专利类型_发明授权', '专利类型_外观设计', '专利类型_实用新型',
       '授权公告日', '申请日', '专利_count'],
      dtype='object')